In [ ]:
import variational_bayes as vb
import numpy as np
from matplotlib import pyplot as plt, rcParams
import scipy.stats
from tqdm import tqdm_notebook
from sklearn import metrics
import multiprocessing
from collections import defaultdict
import pickle
import os
%matplotlib inline

rcParams['figure.dpi'] = 144
rcParams['scatter.marker'] = '.'
np.random.seed(1)

In [ ]:
class IterableAccessor:
    def __init__(self, iterable):
        self._iterable = iterable
        
    def __getitem__(self, y):
        return [item[y] for item in self._iterable]
    
accessor = IterableAccessor([{'a': 1}, {'a': 2}])
accessor['a']

In [ ]:
cov = np.cov(np.random.normal(0, 1, (num_samples//2, size)))
np.testing.assert_allclose(cov, cov.T)

In [ ]:
list_sizes = [20, 50, 100]
for size in list_sizes:
    list_fraction = np.linspace(0.9, 1.1, 21)
    list_num_samples = list_fraction * size
    num_runs = 100
    pd = []
    for num_samples in list_num_samples:
        num_samples = int(num_samples)
        pd.append([vb.is_positive_definite(np.cov(np.random.normal(0, 1, (num_samples, size)), rowvar=False))
                   for _ in range(num_runs)])

    plt.errorbar(list_fraction, np.mean(pd, axis=1), vb.std_mean(pd, axis=1),
                 marker='.', label=str(size))
    
plt.legend()

In [ ]:
num_groups = 3
num_nodes = 50
order = 2

In [ ]:
fractions = 0.025 * 2 ** np.arange(11)
list_num_steps = (num_nodes * (num_nodes * order + 1) * fractions).astype(int)
list_num_steps

In [ ]:
def simulate_parameters():
    # Generate group sizes and groups
    density = np.random.dirichlet(100 * np.ones(num_groups)) # was 10 when things didn't work
    z = np.random.choice(num_groups, num_nodes, p=density)
    onehot = np.zeros((num_nodes, num_groups))
    onehot[np.arange(num_nodes), z] = 1

    # Sample noise precisions for all groups
    noise_precision = np.random.gamma(5000, size=num_groups)
    # noise_precision = 100

    # Sample means and precisions of autoregressive coefficients
    adjacency_mean = np.random.normal(0, 1e-2, size=(num_groups, num_groups, order))
    adjacency_precision = scipy.stats.wishart.rvs(1e5, np.eye(order), size=(num_groups, num_groups))
    if adjacency_precision.ndim < 4:
        adjacency_precision = adjacency_precision.reshape((num_groups, num_groups, 1, 1))

    # Sample the means and precisions of the bias
    bias_mean = np.random.normal(0, 0.1, num_groups)
    bias_precision = np.random.gamma(1e4, 1, num_groups)

    # Sample the matrix of autoregressive coefficients
    cholesky = np.linalg.cholesky(np.linalg.inv(adjacency_precision))
    cholesky = cholesky[z[:, None], z[None, :]]
    adjacency = adjacency_mean[z[:, None], z[None, :]] + \
        np.einsum('...ij,...j', cholesky, np.random.normal(0, 1, (num_nodes, num_nodes, order)))

    # Sample the bias
    bias = np.random.normal(0, 1, num_nodes) / np.sqrt(bias_precision[z]) + bias_mean[z]

    # Construct the coefficients for comparison
    coefficients = vb.pack_coefficients(adjacency, bias)
    
    return {
        'coefficients': coefficients,
        'bias': bias,
        'adjacency': adjacency,
        'z': z,
        'noise_precision': noise_precision,
    }

In [ ]:
def main(*_, tqdm=False):
    result = defaultdict(list)
    parameters = simulate_parameters()
    result['parameters'] = parameters
    
    for num_steps in tqdm_notebook(list_num_steps) if tqdm else list_num_steps:
        series = vb.simulate_series(parameters['bias'], parameters['adjacency'], 
                                    parameters['noise_precision'][parameters['z']], int(num_steps))

        factors = {
            'coefficients': vb.MultiNormalDistribution(
                np.zeros((num_nodes, num_nodes * order + 1)),
                np.ones((num_nodes, 1, 1)) * np.eye(num_nodes * order + 1)
            ),
            'noise_precision': vb.GammaDistribution(
                1e-3 * np.ones(num_nodes),
                1e-3 * np.ones(num_nodes)
            )
        }
        likelihoods = [
            vb.VARDistribution(factors['coefficients'], factors['noise_precision']).likelihood(
                vb.VARDistribution.summary_statistics(series, order)
            ),
            vb.GammaDistribution(1e-6, 1e-6).likelihood(factors['noise_precision']),
            vb.MultiNormalDistribution(
                np.zeros(num_nodes * order + 1), 
                np.eye(num_nodes * order + 1) * 1e-100
            ).likelihood(factors['coefficients'])
        ]

        # Model without hierarchical structure
        model = vb.Model(factors, likelihoods)
        model.update(None, convergence_predicate=1e-3)
        
        # Model with hierarchical structure
        model2 = vb.var_model(series, order, num_groups, shared_noise=False)
        model2.update(None, convergence_predicate=1e-3)
        
        result['means'].append(model['coefficients'].mean)
        result['stds'].append(model['coefficients'].std)
        result['means2'].append(model2['coefficients'].mean)
        result['stds2'].append(model2['coefficients'].std)
        result['zs'].append(model2['z'].mean)
        
        
    result = {key: np.asarray(value) if isinstance(value, list) else value for key, value in result.items()}
    return result

filename = 'var-sample-size.pickle'
if os.path.isfile(filename):
    with open(filename, 'rb') as fp:
        results = pickle.load(fp)
else:
    num_runs = 12
    results = []
    for _ in tqdm_notebook(range(num_runs)):
        results.append(main(tqdm=True))
    with open(filename, 'wb') as fp:
        pickle.dump(results, fp)

results = IterableAccessor(results)

In [ ]:
fig, [(ax1, ax2), (ax3, ax4)] = plt.subplots(2, 2, True)
sigmas = 2

for list_scores, label in zip([np.divide(results['means'], results['stds']),
                               np.divide(results['means2'], results['stds2'])],
                              ['VAR', 'HVAR']):
    significiant = np.mean(np.abs(list_scores) > sigmas, (2, 3))
    ax1.errorbar(fractions, np.mean(significiant, axis=0), np.std(significiant, axis=0) / np.sqrt(num_runs - 1),
                 label=label)
    
list_coefficients = np.asarray(IterableAccessor(results['parameters'])['coefficients'])[:, None]
for residuals in [np.subtract(results['means'], list_coefficients),
                  np.subtract(results['means2'], list_coefficients)]:
    rmse = np.sqrt(np.mean(residuals * residuals, axis=(2, 3)))
    ax4.errorbar(fractions, np.mean(rmse, axis=0), np.std(rmse, axis=0) / np.sqrt(num_runs - 1))
    
list_nmis = []
for parameters, zs in zip(results['parameters'], results['zs']):
    nmis = [metrics.normalized_mutual_info_score(parameters['z'], np.argmax(z, axis=1)) for z in zs]
    list_nmis.append(nmis)
    
ax3.errorbar(fractions, np.mean(list_nmis, axis=0), 
             np.std(list_nmis, axis=0) / np.sqrt(num_runs - 1), color='C1')

for ax in [ax1, ax3, ax4]:
    ax.axvline(1, ls=':', color='k')
    
ax1.set_xscale('log')
ax1.set_ylabel(r'$\mathrm{avg}\left[\left|\frac{\hat\theta}{\sigma}\right| > 2\right]$')
ax1.legend()

ax2.set_axis_off()
ax2.text(0, 1, "$n=%d$\n$p=%d$\n$K=%d$" % (num_nodes, order, num_groups), 
         transform=ax2.transAxes, va='top')

ax3.set_xlabel('Fraction $f$')
ax3.set_ylabel('NMI')

ax4.set_xlabel('Fraction $f$')
ax4.set_ylabel('RMSE')
ax4.set_yscale('log')

fig.tight_layout()